In [2]:
import requests
import websocket
import threading
import time
import json
import queue

import pandas as pd
import numpy as np


In [3]:
def get_unix_time():
    return time.time_ns() // 1000

print(get_unix_time())

1722874478754087


In [4]:
class Client(threading.Thread):
    def __init__(self, url, exchange, interval):
        super().__init__()
        self.ws = websocket.WebSocketApp(
            url=url,
            on_message=self.on_message,
            on_error=self.on_error,
            on_close=self.on_close,
            on_open=self.on_open
        )

        self.exchange = exchange
        self.interval = interval
        self.stop_event = threading.Event()

    def run(self):
        self.ws.run_forever(ping_interval = self.interval)

    def close(self):
        self.stop_event.set()
        if self.ws:
            self.ws.close()

    def on_message(self, ws, message):
        print(message)

    def on_error(self, ws, error):
        print(f"{self.exchange} Error: {error}")

    def on_close(self, ws, error_status, error_message):
        print(f"{self.exchange} Closed. Error Status: {error_status}. Error Message: {error_message}")

    def on_open(self, ws):
        print(f'Connected to {self.exchange}\n')

In [5]:
class SymbolWatcher(Client):
    def __init__(self, symbol, depth, interval):
        url = f"wss://stream.binance.us:9443/ws/{symbol}@depth{depth}"
        self.message_buffer = queue.Queue()
        self.symbol = symbol
        super().__init__(url, "Binance", interval)
        print("watcher created successfully")

    def on_message(self, ws, message):
        self.message_buffer.put(message)
        print("message received")
    
    def write_to_file(self):
        with open(f"{self.symbol}.txt", 'a') as file:
            while not self.stop_event.is_set() or not self.message_buffer.empty():
                try:
                    message = self.message_buffer.get(timeout=1)
                    data = json.loads(message)
                    bids = data["bids"]
                    asks = data["asks"]
                    for i in range(min(len(bids), len(asks))):
                        entry = f"{data['lastUpdateId']}, {get_unix_time()}, {bids[i][0]}, {asks[i][0]}, {bids[i][1]}, {asks[i][1]}\n"
                        file.write(entry)
                        print("data logged")
                except queue.Empty:
                    continue
                except json.JSONDecodeError as e:
                    print(f"JSON decode error: {e}")
                except Exception as e:
                    print(f"Unexpected error: {e}")

In [8]:
ws_client_1 = SymbolWatcher("usdcusdt",10,1000000)
ws_client_1_writer = threading.Thread(target=ws_client_1.write_to_file)

ws_client_1.start()
ws_client_1_writer.start()

watcher created successfully


Connected to Binance

message receiveddata logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged

message receiveddata logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged

message receiveddata logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged

message receiveddata logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged

message receiveddata logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged

message receiveddata logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged

message receiveddata logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged
data logged

message receiveddat

In [9]:
ws_client_1.close()
ws_client_1.join()
ws_client_1_writer.join()

Binance Closed. Error Status: None. Error Message: None


In [10]:
import pandas as pd

df = pd.read_csv('usdcusdt.txt', delim_whitespace=True, header=None, names=['TIMEID', 'UNIXTIME', 'BID', 'ASK', 'BIDSIZE', 'ASKSIZE'])

columns_and_files = {
    'TIMEID': 'timeid.txt',
    'UNIXTIME': 'time.txt',
    'BID': 'bids.txt',
    'BIDSIZE': 'bid_size.txt',
    'ASK': 'asks.txt',
    'ASKSIZE': 'ask_size.txt'
}

for column, filename in columns_and_files.items():
    with open(filename, 'w') as file:
        for d in df[column]:
            file.write(f"{d}\n")

/var/folders/_3/hpp2kxf54pndqxk7bcfbzl6w0000gn/T/ipykernel_30170/3318695232.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv('usdcusdt.txt', delim_whitespace=True, header=None,
